# LSTM Training

In [10]:
import tensorflow as tf
from tensorflow.contrib import rnn

In [2]:
import pandas as pd
import numpy as np
from numpy import *
import itertools
import os
import random

In [3]:
# read finished Data; case training
def readData(src, noConfFlag=True):
    dataseries = []
    for i in itertools.count():
        try:
            #print(i)
            dataseries.append(np.array(pd.read_json(src+ '/'+os.path.basename(src[:-4])+'_'+str(i).zfill(12)+'_keypoints.json').iloc[0,0]['pose_keypoints_2d']))
            if noConfFlag:
                dataseries[-1] = np.append(dataseries[-1][:2],np.delete(dataseries[-1][2:], slice(None, None, 3)))
        except ValueError:
            break
        except IndexError:
            if noConfFlag:
                dataseries.append(np.zeros(25*2))
            else:
                dataseries.append(np.zeros(25*3))
    return dataseries,i

In [4]:
labelNames=['rennen','gehen']
dataPaths=['data/output/rennen/','data/output/gehen/']
samplePaths=[]
data=[]
labels=[]
#collect paths to data
dirs0 = os.listdir(dataPaths[0])
for p in dirs0:
    samplePaths.append(dataPaths[0]+p+'0')
dirs1 = os.listdir(dataPaths[1])
for p in dirs1:
    samplePaths.append(dataPaths[1]+p+'1')

#shuffel data
random.shuffle(samplePaths)
#print(samplePaths)
minLength = 75
for smp in samplePaths:
    l = smp[-1:]
    smp = smp[:-1]
    td, i = readData(smp)
    #if i<minLength:
    #    minLength = i
    data.append(td[10:60])
    if l == '0':
        labels.append(np.array([1,0]))
    elif l == '1':
        labels.append(np.array([0,1]))
    else:
        #labels.append(np.array([0,0]))
        print("can't load: ",smp)
    print('.', end="")
        
print("data loaded.")
print("min Length: ", minLength)

.............................................data loaded.
min Length:  75


In [5]:
#define constants

time_steps=50
#hidden LSTM units
num_units=128
# 25*3=75; 25*2=50
n_input=50
#learning rate for adam
learning_rate=0.001

n_classes=2
#size of batch
batch_size=5

In [6]:
#shape for LSTM
unscaledData = np.concatenate(data, axis=0).reshape((-1,time_steps*50))
l = np.concatenate(labels, axis=0).reshape((-1,2))

In [7]:
print(unscaledData.shape,l.shape)
print(unscaledData)
print(l)

(45, 2500) (45, 2)
[[569.512  231.539  587.797  ... 403.814  397.265  399.905 ]
 [146.666  247.2    133.638  ... 432.473  285.032  431.199 ]
 [  0.       0.       0.     ... 403.805  351.642  382.89  ]
 ...
 [  0.       0.       0.     ... 412.918  226.313  423.37  ]
 [  0.       0.       0.     ...   0.     223.613  476.801 ]
 [ 43.5748 172.706   17.4589 ... 445.502  367.157  441.574 ]]
[[0 1]
 [0 1]
 [1 0]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [1 0]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [1 0]
 [1 0]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [1 0]
 [0 1]
 [0 1]
 [0 1]
 [1 0]
 [0 1]
 [1 0]
 [0 1]
 [0 1]]


In [8]:
#naiv standardize
#minval = np.min(unscaledData[np.nonzero(unscaledData)])
#maxval = np.max(unscaledData[np.nonzero(unscaledData)])
#print(minval,maxval)
#d = (unscaledData-minval)/(maxval-minval)
#minval = np.min(d[np.nonzero(d)])
#maxval = np.max(d[np.nonzero(d)])
#print(minval,maxval)

In [11]:
scaledData = []
unscaledData[unscaledData == 0] = np.NaN
for video in unscaledData:
    temp =[]
    for i in range(0,n_input):
        p=50
        frame = video[p*i:p*(i+1)]
        # replace 0 with x_avg or y_avg
        x_werte = frame[::2]
        y_werte = frame[1::2]
        x_avg = np.nanmean(x_werte)
        y_avg = np.nanmean(y_werte)
        x_werte[isnan(x_werte)] = x_avg
        y_werte[isnan(y_werte)] = y_avg
        # noramlize data with min max
        minval = np.min(frame)
        maxval = np.max(frame)
        temp.append((frame - minval)/(maxval-minval))
    scaledData.append(temp)

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:11: RuntimeWarning: Mean of empty slice
  # This is added back by InteractiveShellApp.init_path()
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:12: RuntimeWarning: Mean of empty slice
  if sys.path[0] == '':


In [12]:
d = np.nan_to_num(scaledData)
print(d)

[[[0.89436546 0.01371373 0.94201044 ... 0.45566945 0.97615797 0.45929917]
  [0.89988243 0.01710504 0.93802661 ... 0.46210814 0.98978031 0.46561158]
  [0.89155268 0.01406681 0.94042763 ... 0.46494947 1.         0.46854653]
  ...
  [0.72436006 0.01378702 0.83438141 ... 0.90331649 0.98593291 0.85506458]
  [0.71719018 0.00723324 0.82747999 ... 0.91704224 0.93785433 0.86882942]
  [0.70813364 0.01378213 0.81931095 ... 0.90263142 0.86779576 0.88183854]]

 [[0.157336   0.48514391 0.11485602 ... 0.97444289 0.00403475 0.93190749]
  [0.15633619 0.47813878 0.11729045 ... 0.98238245 0.         0.93468616]
  [0.14218439 0.46237848 0.10645101 ... 0.99087427 0.         0.93765898]
  ...
  [0.22043796 0.15759678 0.15756787 ... 0.96851437 0.31425887 1.        ]
  [0.26087081 0.16798888 0.1741423  ... 0.98749804 0.31060739 1.        ]
  [0.26822874 0.17665394 0.19499133 ... 1.         0.31098795 0.99404642]]

 [[0.         0.         0.         ... 0.         0.         0.        ]
  [0.         0.      

In [41]:
#split data in train and testset
trainSize=30 

train_data = d[:trainSize,:]
train_labels = l[:trainSize,:]
test_data = d[trainSize:,:]
test_labels = l[trainSize:,:]

In [42]:
test_data.shape, test_labels.shape

((15, 50, 50), (15, 2))

In [43]:
train_data.shape, train_labels.shape

((30, 50, 50), (30, 2))

In [44]:
#weights and biases of appropriate shape to accomplish above task
out_weights=tf.Variable(tf.random_normal([num_units,n_classes]))
out_bias=tf.Variable(tf.random_normal([n_classes]))

#defining placeholders
#input image placeholder
x=tf.placeholder("float",[None,time_steps,n_input])
#input label placeholder
y=tf.placeholder("float",[None,n_classes])

In [45]:
#processing the input tensor from [batch_size,n_steps,n_input] to "time_steps" number of [batch_size,n_input] tensors
input=tf.unstack(x ,time_steps,1)

In [46]:
#defining the network
lstm_layer=tf.nn.rnn_cell.LSTMCell(num_units,forget_bias=1)
outputs,_=rnn.static_rnn(lstm_layer,input,dtype="float32")

ValueError: Variable rnn/lstm_cell/kernel already exists, disallowed. Did you mean to set reuse=True or reuse=tf.AUTO_REUSE in VarScope? Originally defined at:

  File "<ipython-input-27-38ec1eb511c1>", line 3, in <module>
    outputs,_=rnn.static_rnn(lstm_layer,input,dtype="float32")
  File "/usr/local/lib/python3.5/dist-packages/IPython/core/interactiveshell.py", line 3267, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/usr/local/lib/python3.5/dist-packages/IPython/core/interactiveshell.py", line 3185, in run_ast_nodes
    if (yield from self.run_code(code, result)):


In [47]:
#converting last output of dimension [batch_size,num_units] to [batch_size,n_classes] by out_weight multiplication
prediction=tf.matmul(outputs[-1],out_weights)+out_bias

In [48]:
#loss_function
loss=tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=prediction,labels=y))
#optimization
opt=tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(loss)

#model evaluation
correct_prediction=tf.equal(tf.argmax(prediction,1),tf.argmax(y,1))
accuracy=tf.reduce_mean(tf.cast(correct_prediction,tf.float32))

ValueError: Variable rnn/lstm_cell/kernel/Adam/ already exists, disallowed. Did you mean to set reuse=True or reuse=tf.AUTO_REUSE in VarScope? Originally defined at:

  File "<ipython-input-29-bf440bc74171>", line 4, in <module>
    opt=tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(loss)
  File "/usr/local/lib/python3.5/dist-packages/IPython/core/interactiveshell.py", line 3267, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/usr/local/lib/python3.5/dist-packages/IPython/core/interactiveshell.py", line 3185, in run_ast_nodes
    if (yield from self.run_code(code, result)):


In [49]:
#initialize variables
init=tf.global_variables_initializer()
with tf.Session() as sess:
    sess.run(init)
    iter=0
    while iter<trainSize//batch_size:
        print(iter)
        batch_x=train_data[batch_size*iter:batch_size*(iter+1)]
        
        batch_y=train_labels[batch_size*iter:batch_size*(iter+1)]
        #print(batch_x)
        batch_x=batch_x.reshape((batch_size,time_steps,n_input))

        sess.run(opt, feed_dict={x: batch_x, y: batch_y})

        if iter%5==0 and iter!=0:
            acc=sess.run(accuracy,feed_dict={x:batch_x,y:batch_y})
            los=sess.run(loss,feed_dict={x:batch_x,y:batch_y})
            print("For iter ",iter)
            print("Accuracy ",acc)
            print("Loss ",los)
            print("__________________")

        iter=iter+1

    #calculating test accuracy
    test_data = test_data[:batch_size].reshape((-1, time_steps, n_input))
    test_label = test_labels[:batch_size]
    print("Testing Accuracy:", sess.run(accuracy, feed_dict={x: test_data, y: test_label}))

0


InvalidArgumentError: You must feed a value for placeholder tensor 'Placeholder' with dtype float and shape [?,50,50]
	 [[node Placeholder (defined at <ipython-input-25-490822d444fd>:7)  = Placeholder[dtype=DT_FLOAT, shape=[?,50,50], _device="/job:localhost/replica:0/task:0/device:CPU:0"]()]]

Caused by op 'Placeholder', defined at:
  File "/usr/lib/python3.5/runpy.py", line 184, in _run_module_as_main
    "__main__", mod_spec)
  File "/usr/lib/python3.5/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/usr/local/lib/python3.5/dist-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/usr/local/lib/python3.5/dist-packages/ipykernel/kernelapp.py", line 505, in start
    self.io_loop.start()
  File "/usr/local/lib/python3.5/dist-packages/tornado/platform/asyncio.py", line 132, in start
    self.asyncio_loo

In [20]:
sess.close()

# Visualisation

In [21]:

show_data = unscaledData.reshape(-1,n_input)
show_label = [np.where(r==1)[0][0] for r in l]

In [22]:
#show_label

In [23]:
show_data.shape

(2250, 50)

In [24]:
testing = show_data[1]
testing[1]

nan

In [25]:
pairs = [
        (1,8),(1,2),(1,5),(2,3),(3,4),   
        (5,6),(6,7),(8,9),(9,10),(10,11),
        (8,12),(12,13),(13,14),(1,0),(0,15), 
        (15,17),(0,16),(16,18),
        (14,19),(19,20),(14,21),(11,22),(22,23),
        (11,24)
        ]

In [26]:
pairs[0][1]

8

In [27]:
import pygame
import sys, traceback

# Initialize Pygame.
pygame.init()
# Set size of pygame window.
screen=pygame.display.set_mode((900,750))
# Create empty pygame surface.
background = pygame.Surface(screen.get_size())
# Fill the background white color.
background.fill((255, 255, 255))
# Convert Surface object to make blitting faster.
background = background.convert()
# Copy background to screen (position (0, 0) is upper left corner).
screen.blit(background, (0,0))
# Create Pygame clock object.
clock = pygame.time.Clock()

mainloop = True
# Desired framerate in frames per second. Try out other values.
FPS = 30
# How many seconds the "game" is played.
playtime = 0.0

count = 0.0

speed = 1

try:
    
    while mainloop:
        # Do not go faster than this framerate.
        milliseconds = clock.tick(FPS*speed)
        playtime += milliseconds / 1000.0

        for event in pygame.event.get():
            # User presses QUIT-button.
            if event.type == pygame.QUIT:
                mainloop = False
            elif event.type == pygame.KEYDOWN:
                # User presses ESCAPE-Key
                if event.key == pygame.K_ESCAPE:
                    mainloop = False

        # Print framerate and playtime in titlebar.
        currentSample=int(count/n_input)%len(show_label)
        text = "FPS: {0:.2f}   Playtime: {1:.2f}  NR: {2:d}  Label: ".format(clock.get_fps(), playtime, currentSample+1)
        text += labelNames[show_label[currentSample]]
        pygame.display.set_caption(text)
        background.fill((255, 255, 255))
        f = show_data[int(count)%(show_data.shape[0])]
        
        for p in pairs:
            x1=int(f[p[0]*2])
            y1=int(f[p[0]*2+1])
            x2=int(f[p[1]*2])
            y2=int(f[p[1]*2+1])
            if not (x1 == 0 or x2 == 0 or y1 == 0 or y2 == 0):
                pygame.draw.line(background, (0, 0, 0), [x1,y1], [x2,y2], 1)
        for i in range(0,f.shape[0],2):#2 for no conf
            #confColor = 255 - int(255*f[i+2])%255
            #pygame.draw.rect(background, (confColor, confColor, confColor), (int(f[i]),int(f[i+1]),4,4)) #bw
            #pygame.draw.rect(background, (confColor, 0, 255-confColor), (int(f[i]),int(f[i+1]),4,4)) #rb
            if i!=25:
                pygame.draw.rect(background, (0, 0, 0), (int(f[i]),int(f[i+1]),4,4)) #no Conf
            else:
                pygame.draw.rect(background, (255, 0, 0), (int(f[i]),int(f[i+1]),4,4)) #no Conf
        screen.blit(background, (300-f[8*2],300-f[8*2+1]))#2 for no conf
        #screen.blit(background, (0,0))
        #Update Pygame display.
        pygame.display.update()

        count += 1
except TypeError:
    print("Type error: ", count," ", i)
    traceback.print_exc(file=sys.stdout)
except:
    traceback.print_exc(file=sys.stdout)
else:
    pygame.quit()
# Finish Pygame.
pygame.quit()

# At the very last:
print("This game was played for {0:.2f} seconds".format(playtime))

ImportError: No module named 'pygame'

In [ ]:
pygame.display.list_modes()

# Live Data-read

In [ ]:
import sys
import time
import logging
from watchdog.observers import Observer
from watchdog.events import LoggingEventHandler
from watchdog.events import PatternMatchingEventHandler  

In [ ]:
class MyHandler(PatternMatchingEventHandler):
    patterns = ["*.json"]
    data = []

    def process(self, event):
        """
        event.event_type 
            'modified' | 'created' | 'moved' | 'deleted'
        event.is_directory
            True | False
        event.src_path
            path/to/observed/file
        """
        # the file will be processed there
        try:
            #print(i)
            f = pd.read_json(event.src_path).iloc[0,0]['pose_keypoints_2d']
            self.data.append(f)
            print(f)
        except ValueError:
            print("ERROR reading: ",event.src_path, event.event_type)


    def on_created(self, event):
        self.process(event)
        
observer = Observer()
observer.schedule(MyHandler(), '.')
observer.start()

try:
    while True:
        time.sleep(1)
except KeyboardInterrupt:
    observer.stop()
observer.join()